In [5]:
import pandas as pd
import os

# --- 設定項目 ---
# 監査対象のCSVファイル
RESULTS_CSV_PATH = '../data/processed/citing_papers_with_paths.csv'

# --- 1. データの読み込み ---
try:
    df_results = pd.read_csv(RESULTS_CSV_PATH)
    print(f"'{RESULTS_CSV_PATH}' を正常に読み込みました。")
    print(f"記録されている論文総数: {len(df_results)}件")
except FileNotFoundError:
    print(f"エラー: '{RESULTS_CSV_PATH}' が見つかりません。")
    df_results = pd.DataFrame()

if not df_results.empty:
    
    # --- 2. ダウンロード結果のサマリー ---
    print("\n--- ① ダウンロード結果のサマリー ---")
    
    # download_status列の各項目の数を集計して表示
    status_counts = df_results['download_status'].value_counts()
    print(status_counts)
    
    
    # --- 3. 保存されたXMLファイルの存在確認 ---
    print("\n\n--- ② 保存されたXMLファイルの整合性チェック ---")
    
    # ダウンロードに成功した（と記録されている）論文に絞り込む
    df_success = df_results[
        df_results['download_status'].str.startswith('success', na=False)
    ].copy()

    if not df_success.empty:
        print(f"ダウンロード成功と記録されている {len(df_success)}件について、ファイルが存在するか確認します...")
        
        # 'fulltext_xml_path'列のパスが実際に存在するかをチェックし、結果を新しい列に記録
        # os.path.exists()はパスが存在すればTrue, なければFalseを返す
        df_success['file_exists'] = df_success['fulltext_xml_path'].apply(
            lambda path: os.path.exists(path) if isinstance(path, str) else False
        )
        
        # ファイル存在チェックの結果を集計
        existence_counts = df_success['file_exists'].value_counts()
        print("\n[チェック結果]")
        print(existence_counts)
        
        # もし存在しないファイル（False）があれば、そのリストを表示
        missing_files = df_success[df_success['file_exists'] == False]
        
        if not missing_files.empty:
            print("\n【警告】以下の論文は成功と記録されていますが、XMLファイルが見つかりませんでした：")
            print(missing_files[['citing_paper_doi', 'fulltext_xml_path', 'download_status']])
        else:
            print("\n[OK] 全ての成功レコードについて、ファイルが正しく存在することを確認しました。")
            
    else:
        print("ダウンロードに成功した論文はまだありません。")

'../data/processed/citing_papers_with_paths.csv' を正常に読み込みました。
記録されている論文総数: 136703件

--- ① ダウンロード結果のサマリー ---
download_status
failed (Status: 404)    95983
success (cached)        18336
success (on retry)      14798
success (downloaded)     7586
Name: count, dtype: int64


--- ② 保存されたXMLファイルの整合性チェック ---
ダウンロード成功と記録されている 40720件について、ファイルが存在するか確認します...

[チェック結果]
file_exists
True    40720
Name: count, dtype: int64

[OK] 全ての成功レコードについて、ファイルが正しく存在することを確認しました。
